In [1]:
from PIL import Image
import requests
from io import BytesIO
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
from skimage.transform import rescale, resize, downscale_local_mean
import random
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob
import random
import shutil

In [ ]:
typs = ["Baso","eosi","lymp","mixt","mono","neut"]
class_id = "0"
count = 1
for ty in typs:
    gts = glob.glob("GroundTruthSegmentation/"+ty+"/areaforexpert1/*.jpg")
    main_img_pth = "MainDataset/"+ty+"/"
    for gt in gts:
        gt_image = gt.split("/")[-1]
        main_name = gt_image.split("_")[0]
        txt_name = "{:010d}.txt".format(int(count))
        img_name = "{:010d}.jpg".format(int(count))
        print(img_name)
        img_main = imread(main_img_pth + main_name + ".bmp")
        new_img = resize(img_main,(1200,1600,3),anti_aliasing=True)
        imsave(os.path.join("images/",img_name),new_img)
        img = imread(gt)
        h, w = img.shape
        thresh = threshold_otsu(img)
        bw = img > thresh
        label_image = label(bw)
        result = ""
        for region in regionprops(label_image):
            minr, minc, maxr, maxc = region.bbox
            x1 = minc / w
            y1 = minr / h
            bw = (maxc - minc) / w
            bh = (maxr - minr) / h
            xc = x1 + (bw / 2)
            yc = y1 + (bh / 2)
            result += "\n" + class_id + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh) 
        f = open("labels/" + txt_name,'w')
        result = result[1:]
        f.write(result)
        f.close()
        count += 1

In [2]:
count = 251
files = glob.glob("labels/*.txt")
degs = [45,90,135,180,225,270,315]
for file in files:
    print(file)
    f = open(file,"r")
    lines = list(f)
    f.close()
    rects = []
    img_name = file.split("/")[1].replace(".txt",".jpg")
    for line in lines:
        class_id, xc, yc, bw, bh = line.split(" ")
        xc = float(xc)
        yc = float(yc)
        bw = float(bw)
        bh = float(bh)
        minx = (xc - (bw/2)) * 1600
        miny = (yc - (bh/2)) * 1200
        bw = bw * 1600
        bh = bh * 1200
        maxx = minx + bw
        maxy = miny + bh
        rects.append(BoundingBox(x1=int(minx),y1=int(miny),x2=int(maxx),y2=int(maxy),label=class_id))
    img = Image.open("images/"+img_name)
    img = np.array(img)
    bbs = BoundingBoxesOnImage(rects,shape=img.shape)
    for deg in degs:
        name = "{:010d}".format(count)
        image_aug, bbs_aug = iaa.Affine(rotate=deg,mode="reflect")(image=img, bounding_boxes=bbs)
        aug_h, aug_w, aug_c = image_aug.shape
        clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
        bboxs = ""
        for bb in clip_off_bbs.bounding_boxes:
            x1 = bb.x1
            y1 = bb.y1
            x2 = bb.x2
            y2 = bb.y2
            bb_w = (x2-x1)/aug_w
            bb_h = (y2-y1)/aug_h
            bb_xc = (x1/aug_w) + (bb_w/2)
            bb_yc = (y1/aug_h) + (bb_h/2)
            bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
        bboxs = bboxs[1:]
        im_aug = Image.fromarray(image_aug)
        im_aug.save(os.path.join('images',name+".jpg"))
        f = open(os.path.join('labels',name+".txt"),'w')
        f.write(bboxs)
        f.close()
        print("{} {} {}".format(count,deg,os.path.join('labels',name+".txt")))
        count += 1

labels/0000000001.txt
251 45 labels/0000000251.txt
252 90 labels/0000000252.txt
253 135 labels/0000000253.txt
254 180 labels/0000000254.txt
255 225 labels/0000000255.txt
256 270 labels/0000000256.txt
257 315 labels/0000000257.txt
labels/0000000002.txt
258 45 labels/0000000258.txt
259 90 labels/0000000259.txt
260 135 labels/0000000260.txt
261 180 labels/0000000261.txt
262 225 labels/0000000262.txt
263 270 labels/0000000263.txt
264 315 labels/0000000264.txt
labels/0000000003.txt
265 45 labels/0000000265.txt
266 90 labels/0000000266.txt
267 135 labels/0000000267.txt
268 180 labels/0000000268.txt
269 225 labels/0000000269.txt
270 270 labels/0000000270.txt
271 315 labels/0000000271.txt
labels/0000000004.txt
272 45 labels/0000000272.txt
273 90 labels/0000000273.txt
274 135 labels/0000000274.txt
275 180 labels/0000000275.txt
276 225 labels/0000000276.txt
277 270 labels/0000000277.txt
278 315 labels/0000000278.txt
labels/0000000005.txt
279 45 labels/0000000279.txt
280 90 labels/0000000280.txt


503 45 labels/0000000503.txt
504 90 labels/0000000504.txt
505 135 labels/0000000505.txt
506 180 labels/0000000506.txt
507 225 labels/0000000507.txt
508 270 labels/0000000508.txt
509 315 labels/0000000509.txt
labels/0000000038.txt
510 45 labels/0000000510.txt
511 90 labels/0000000511.txt
512 135 labels/0000000512.txt
513 180 labels/0000000513.txt
514 225 labels/0000000514.txt
515 270 labels/0000000515.txt
516 315 labels/0000000516.txt
labels/0000000039.txt
517 45 labels/0000000517.txt
518 90 labels/0000000518.txt
519 135 labels/0000000519.txt
520 180 labels/0000000520.txt
521 225 labels/0000000521.txt
522 270 labels/0000000522.txt
523 315 labels/0000000523.txt
labels/0000000040.txt
524 45 labels/0000000524.txt
525 90 labels/0000000525.txt
526 135 labels/0000000526.txt
527 180 labels/0000000527.txt
528 225 labels/0000000528.txt
529 270 labels/0000000529.txt
530 315 labels/0000000530.txt
labels/0000000041.txt
531 45 labels/0000000531.txt
532 90 labels/0000000532.txt
533 135 labels/0000000

755 45 labels/0000000755.txt
756 90 labels/0000000756.txt
757 135 labels/0000000757.txt
758 180 labels/0000000758.txt
759 225 labels/0000000759.txt
760 270 labels/0000000760.txt
761 315 labels/0000000761.txt
labels/0000000074.txt
762 45 labels/0000000762.txt
763 90 labels/0000000763.txt
764 135 labels/0000000764.txt
765 180 labels/0000000765.txt
766 225 labels/0000000766.txt
767 270 labels/0000000767.txt
768 315 labels/0000000768.txt
labels/0000000075.txt
769 45 labels/0000000769.txt
770 90 labels/0000000770.txt
771 135 labels/0000000771.txt
772 180 labels/0000000772.txt
773 225 labels/0000000773.txt
774 270 labels/0000000774.txt
775 315 labels/0000000775.txt
labels/0000000076.txt
776 45 labels/0000000776.txt
777 90 labels/0000000777.txt
778 135 labels/0000000778.txt
779 180 labels/0000000779.txt
780 225 labels/0000000780.txt
781 270 labels/0000000781.txt
782 315 labels/0000000782.txt
labels/0000000077.txt
783 45 labels/0000000783.txt
784 90 labels/0000000784.txt
785 135 labels/0000000

1006 315 labels/0000001006.txt
labels/0000000109.txt
1007 45 labels/0000001007.txt
1008 90 labels/0000001008.txt
1009 135 labels/0000001009.txt
1010 180 labels/0000001010.txt
1011 225 labels/0000001011.txt
1012 270 labels/0000001012.txt
1013 315 labels/0000001013.txt
labels/0000000110.txt
1014 45 labels/0000001014.txt
1015 90 labels/0000001015.txt
1016 135 labels/0000001016.txt
1017 180 labels/0000001017.txt
1018 225 labels/0000001018.txt
1019 270 labels/0000001019.txt
1020 315 labels/0000001020.txt
labels/0000000111.txt
1021 45 labels/0000001021.txt
1022 90 labels/0000001022.txt
1023 135 labels/0000001023.txt
1024 180 labels/0000001024.txt
1025 225 labels/0000001025.txt
1026 270 labels/0000001026.txt
1027 315 labels/0000001027.txt
labels/0000000112.txt
1028 45 labels/0000001028.txt
1029 90 labels/0000001029.txt
1030 135 labels/0000001030.txt
1031 180 labels/0000001031.txt
1032 225 labels/0000001032.txt
1033 270 labels/0000001033.txt
1034 315 labels/0000001034.txt
labels/0000000113.txt

1249 225 labels/0000001249.txt
1250 270 labels/0000001250.txt
1251 315 labels/0000001251.txt
labels/0000000144.txt
1252 45 labels/0000001252.txt
1253 90 labels/0000001253.txt
1254 135 labels/0000001254.txt
1255 180 labels/0000001255.txt
1256 225 labels/0000001256.txt
1257 270 labels/0000001257.txt
1258 315 labels/0000001258.txt
labels/0000000145.txt
1259 45 labels/0000001259.txt
1260 90 labels/0000001260.txt
1261 135 labels/0000001261.txt
1262 180 labels/0000001262.txt
1263 225 labels/0000001263.txt
1264 270 labels/0000001264.txt
1265 315 labels/0000001265.txt
labels/0000000146.txt
1266 45 labels/0000001266.txt
1267 90 labels/0000001267.txt
1268 135 labels/0000001268.txt
1269 180 labels/0000001269.txt
1270 225 labels/0000001270.txt
1271 270 labels/0000001271.txt
1272 315 labels/0000001272.txt
labels/0000000147.txt
1273 45 labels/0000001273.txt
1274 90 labels/0000001274.txt
1275 135 labels/0000001275.txt
1276 180 labels/0000001276.txt
1277 225 labels/0000001277.txt
1278 270 labels/00000

1494 225 labels/0000001494.txt
1495 270 labels/0000001495.txt
1496 315 labels/0000001496.txt
labels/0000000179.txt
1497 45 labels/0000001497.txt
1498 90 labels/0000001498.txt
1499 135 labels/0000001499.txt
1500 180 labels/0000001500.txt
1501 225 labels/0000001501.txt
1502 270 labels/0000001502.txt
1503 315 labels/0000001503.txt
labels/0000000180.txt
1504 45 labels/0000001504.txt
1505 90 labels/0000001505.txt
1506 135 labels/0000001506.txt
1507 180 labels/0000001507.txt
1508 225 labels/0000001508.txt
1509 270 labels/0000001509.txt
1510 315 labels/0000001510.txt
labels/0000000181.txt
1511 45 labels/0000001511.txt
1512 90 labels/0000001512.txt
1513 135 labels/0000001513.txt
1514 180 labels/0000001514.txt
1515 225 labels/0000001515.txt
1516 270 labels/0000001516.txt
1517 315 labels/0000001517.txt
labels/0000000182.txt
1518 45 labels/0000001518.txt
1519 90 labels/0000001519.txt
1520 135 labels/0000001520.txt
1521 180 labels/0000001521.txt
1522 225 labels/0000001522.txt
1523 270 labels/00000

1740 270 labels/0000001740.txt
1741 315 labels/0000001741.txt
labels/0000000214.txt
1742 45 labels/0000001742.txt
1743 90 labels/0000001743.txt
1744 135 labels/0000001744.txt
1745 180 labels/0000001745.txt
1746 225 labels/0000001746.txt
1747 270 labels/0000001747.txt
1748 315 labels/0000001748.txt
labels/0000000215.txt
1749 45 labels/0000001749.txt
1750 90 labels/0000001750.txt
1751 135 labels/0000001751.txt
1752 180 labels/0000001752.txt
1753 225 labels/0000001753.txt
1754 270 labels/0000001754.txt
1755 315 labels/0000001755.txt
labels/0000000216.txt
1756 45 labels/0000001756.txt
1757 90 labels/0000001757.txt
1758 135 labels/0000001758.txt
1759 180 labels/0000001759.txt
1760 225 labels/0000001760.txt
1761 270 labels/0000001761.txt
1762 315 labels/0000001762.txt
labels/0000000217.txt
1763 45 labels/0000001763.txt
1764 90 labels/0000001764.txt
1765 135 labels/0000001765.txt
1766 180 labels/0000001766.txt
1767 225 labels/0000001767.txt
1768 270 labels/0000001768.txt
1769 315 labels/00000

1982 135 labels/0000001982.txt
1983 180 labels/0000001983.txt
1984 225 labels/0000001984.txt
1985 270 labels/0000001985.txt
1986 315 labels/0000001986.txt
labels/0000000249.txt
1987 45 labels/0000001987.txt
1988 90 labels/0000001988.txt
1989 135 labels/0000001989.txt
1990 180 labels/0000001990.txt
1991 225 labels/0000001991.txt
1992 270 labels/0000001992.txt
1993 315 labels/0000001993.txt
labels/0000000250.txt
1994 45 labels/0000001994.txt
1995 90 labels/0000001995.txt
1996 135 labels/0000001996.txt
1997 180 labels/0000001997.txt
1998 225 labels/0000001998.txt
1999 270 labels/0000001999.txt
2000 315 labels/0000002000.txt


In [3]:
# clear empty box
files = glob.glob("images/*")
count = 0
for file in files:
    img = file.split("/")[-1]
    name = img.split('.')[0]
    f = open("labels/"+name+".txt","r")
    txt = list(f)
    f.close()
    if len(txt) == 0:
        print(img)
        os.remove("labels/"+name+".txt")
        os.remove(file)
        count += 1
print(count)

0000000252.jpg
0000000256.jpg
0000000279.jpg
0000000280.jpg
0000000283.jpg
0000000284.jpg
0000000308.jpg
0000000309.jpg
0000000312.jpg
0000000313.jpg
0000000342.jpg
0000000343.jpg
0000000346.jpg
0000000347.jpg
0000000370.jpg
0000000371.jpg
0000000372.jpg
0000000374.jpg
0000000375.jpg
0000000376.jpg
0000000377.jpg
0000000378.jpg
0000000379.jpg
0000000381.jpg
0000000382.jpg
0000000383.jpg
0000000391.jpg
0000000392.jpg
0000000393.jpg
0000000395.jpg
0000000396.jpg
0000000397.jpg
0000000405.jpg
0000000406.jpg
0000000407.jpg
0000000409.jpg
0000000410.jpg
0000000411.jpg
0000000440.jpg
0000000441.jpg
0000000444.jpg
0000000445.jpg
0000000462.jpg
0000000463.jpg
0000000466.jpg
0000000467.jpg
0000000468.jpg
0000000472.jpg
0000000482.jpg
0000000483.jpg
0000000486.jpg
0000000487.jpg
0000000497.jpg
0000000498.jpg
0000000501.jpg
0000000502.jpg
0000000511.jpg
0000000512.jpg
0000000515.jpg
0000000516.jpg
0000000547.jpg
0000000551.jpg
0000000692.jpg
0000000693.jpg
0000000696.jpg
0000000697.jpg
0000000762

In [ ]:
# resize image
gts = glob.glob("GroundTruthSegmentation/neut/areaforexpert1/*.bmp")

for gt in gts:
    img = imread(gt)
    new_img = resize(img,(1200,1600),anti_aliasing=True)
    new_name = gt.split(".")[0] + ".jpg"
    imsave(new_name,new_img)
    print(new_name)